In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
from tqdm import tqdm
from skimage import transform 
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
from segment_anything import sam_model_registry, build_sam_vit_b_multiclass
import random
from datetime import datetime
import pandas as pd
import nibabel as nib
import pickle   

In [2]:
listo = []
for i in range(1, 103):
    label_number = i
    df1 = pd.read_csv(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/per_class_isolated_df/yolov7/conf_thresh_010/path_df_label{label_number}_only_with_bbox_yolov7.csv', index_col=0)
    df1['label_number'] = label_number
    listo.append(df1)

pooled_label_df = pd.concat(listo, ignore_index=True, axis=0).reset_index()
pooled_label_df


,index,id,slice,image_embedding_slice_path,segmentation_slice_path,image_path,bbox_0,bbox_1,bbox_2,bbox_3,label_number
0,0,965367,155,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/965367/mr...,60,128,213,187,1
1,1,295146,113,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/295146/mr...,70,131,189,176,1
2,2,529953,150,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/529953/mr...,52,128,206,186,1
3,3,362034,149,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/362034/mr...,55,127,208,185,1
4,4,749361,167,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/749361/mr...,97,130,205,183,1
...,...,...,...,...,...,...,...,...,...,...,...
4095076,4095076,877168,156,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/877168/mr...,130,87,163,103,102
4095077,4095077,102513,159,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/102513/mr...,136,86,165,118,102
4095078,4095078,517239,134,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/517239/mr...,121,86,169,98,102
4095079,4095079,119833,152,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/119833/mr...,128,85,169,100,102


In [4]:
asdf = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/per_class_isolated_df/yolov7/path_df_pooled_labels_only_with_bbox_yolov7.csv')
asdf

,index,id,slice,image_embedding_slice_path,segmentation_slice_path,image_path,bbox_0,bbox_1,bbox_2,bbox_3,label_number
0,0,965367,155,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/965367/mr...,60,128,213,187,1
1,1,295146,113,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/295146/mr...,70,131,189,176,1
2,2,529953,150,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/529953/mr...,52,128,206,186,1
3,3,362034,149,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/362034/mr...,55,127,208,185,1
4,4,749361,167,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/749361/mr...,97,130,205,183,1
...,...,...,...,...,...,...,...,...,...,...,...
3582266,3582266,154532,141,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/154532/mr...,121,88,170,103,102
3582267,3582267,513736,133,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/513736/mr...,121,87,145,95,102
3582268,3582268,102109,162,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/102109/mr...,143,89,161,118,102
3582269,3582269,453441,146,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/453441/mr...,129,86,170,102,102


In [3]:
pooled_label_df.to_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/per_class_isolated_df/yolov7/conf_thresh_010/path_df_pooled_labels_only_with_bbox_yolov7.csv', index=False)

In [2]:
# for test now
listo = []
for i in range(1, 103):
    label_number = i
    df1 = pd.read_csv(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/per_class_isolated_df/yolov7/test/path_df_label{label_number}_only_with_bbox_yolov7_TEST.csv', index_col=0)
    df1['label_number'] = label_number
    listo.append(df1)

pooled_label_df = pd.concat(listo, ignore_index=True, axis=0).reset_index()
pooled_label_df


,index,id,slice,image_embedding_slice_path,segmentation_slice_path,image_path,bbox_0,bbox_1,bbox_2,bbox_3,label_number
0,0,680250,127,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/680250/mr...,59,130,206,185,1
1,1,955465,117,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/955465/mr...,61,130,200,186,1
2,2,627549,109,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/627549/mr...,65,130,189,183,1
3,3,255740,172,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/255740/mr...,120,130,195,180,1
4,4,251833,161,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/251833/mr...,50,127,204,192,1
...,...,...,...,...,...,...,...,...,...,...,...
396846,396846,765056,154,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/765056/mr...,135,89,167,120,102
396847,396847,877168,156,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/877168/mr...,130,87,163,103,102
396848,396848,102513,159,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/102513/mr...,136,86,165,118,102
396849,396849,517239,134,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/luilab/karthik/pediatric_seg_proj/h...,/gpfs/data/cbi/hcp/hcp_seg/data_orig/517239/mr...,121,86,169,98,102
